In [2]:
import ir_datasets
import tqdm
import pyterrier as pt
from pathlib import Path
import re
import pandas as pd
from pyterrier.measures import RR, nDCG, MAP

In [3]:
pt.datasets.list_datasets()

,dataset,topics,topics_lang,qrels,corpus,corpus_lang,index,info_url
0,50pct,"[training, validation]",en,"[training, validation]",None,None,"[ex2, ex3]",None
1,antique,"[train, test]",en,"[train, test]",True,en,None,https://ciir.cs.umass.edu/downloads/Antique/re...
2,vaswani,True,en,True,True,en,True,http://ir.dcs.gla.ac.uk/resources/test_collect...
3,msmarco_document,"[train, dev, test, test-2020, leaderboard-2020]",en,"[train, dev, test, test-2020]",True,en,True,https://microsoft.github.io/msmarco/
4,msmarcov2_document,"[train, dev1, dev2, valid1, valid2, trec_2021]",en,"[train, dev1, dev2, valid1, valid2]",None,None,True,https://microsoft.github.io/msmarco/TREC-Deep-...
...,...,...,...,...,...,...,...,...
763,irds:neuclir,None,None,None,None,None,None,https://ir-datasets.com/neuclir.html
764,irds:neuclir/1,None,None,None,None,None,None,https://ir-datasets.com/neuclir.html#neuclir/1
779,irds:sara,True,en,True,True,en,None,https://ir-datasets.com/sara.html
780,trec-deep-learning-docs,"[train, dev, test, test-2020, leaderboard-2020]",en,"[train, dev, test, test-2020]",True,en,True,https://microsoft.github.io/msmarco/


In [4]:
# Load datasets
msmarco = ir_datasets.load("msmarco-passage/train")
antique = ir_datasets.load("antique/test")

In [5]:
print(msmarco)

Dataset(id='msmarco-passage/train', provides=['docs', 'queries', 'qrels', 'scoreddocs', 'docpairs'])


In [6]:
print(msmarco.docs[:10])

[INFO] [starting] building docstore
docs_iter: 100%|█████████████████| 8841823/8841823 [01:04<00:00, 137107.06doc/s]
[INFO] [finished] docs_iter: [01:04] [8841823doc] [137107.06doc/s]
[INFO] [finished] building docstore [01:05]


In [7]:
def msmarco_gen(limit=100000):
    lastdoc = 0
    for elem in msmarco.docs:
        if lastdoc >= limit:
            break
        yield {
            "docno": elem.doc_id,
            "text": elem.text,
        }
        lastdoc += 1

In [8]:
max(len(text.encode("utf-8")) for _, text in msmarco.docs)

1669

In [20]:
idx_path = Path.cwd() / "indices" / "msmarco_test"
indexer = pt.IterDictIndexer(
    str(idx_path),
    meta={
        "docno": 20,
        "text": 4096,
    },
    stemmer="porter",
    stopwords="terrier",
)

In [21]:
# Dont run this cell if you have already indexed the data
# index_ref = indexer.index(msmarco_gen())

In [10]:
print(antique.docs[0])

[INFO] [starting] building docstore
docs_iter: 100%|███████████████████| 403666/403666 [00:02<00:00, 162761.93doc/s]
[INFO] [finished] docs_iter: [00:02] [403666doc] [162761.93doc/s]
[INFO] [finished] building docstore [2.48s]


GenericDoc(doc_id='2020338_0', text="A small group of politicians believed strongly that the fact that Saddam Hussien remained in power after the first Gulf War was a signal of weakness to the rest of the world, one that invited attacks and terrorism. Shortly after taking power with George Bush in 2000 and after the attack on 9/11, they were able to use the terrorist attacks to justify war with Iraq on this basis and exaggerated threats of the development of weapons of mass destruction. The military strength of the U.S. and the brutality of Saddam's regime led them to imagine that the military and political victory would be relatively easy.")


In [11]:
def antique_gen(limit=100000):
    lastdoc = 0
    for elem in antique.docs:
        if lastdoc >= limit:
            break
        yield {
            "docno": elem.doc_id,
            "text": elem.text,
        }
        lastdoc += 1

In [12]:
max(len(text.encode("utf-8")) for _, text in antique.docs)

4000

In [22]:
idx_path = Path.cwd() / "indices" / "antique_test"
indexer = pt.IterDictIndexer(
    str(idx_path),
    meta={
        "docno": 20,
        "text": 4096,
    },
    stemmer="porter",
    stopwords="terrier",
)

In [23]:
# Don't run this cell if you have already indexed the data
# index_ref = indexer.index(antique_gen())

14:15:38.120 [main] WARN org.terrier.structures.indexing.Indexer -- Adding an empty document to the index (730691_1) - further warnings are suppressed
14:15:56.439 [main] WARN org.terrier.structures.indexing.Indexer -- Indexed 502 empty documents


In [24]:
# Define index paths
index_dir_msmarco = Path.cwd() / "indices" / "msmarco_test"
index_dir_antique = Path.cwd() / "indices" / "antique_test"

# Load the indexes
index_msmarco = pt.IndexFactory.of(str(index_dir_msmarco))
index_antique = pt.IndexFactory.of(str(index_dir_antique))

# Use BM25 as the baseline retriever
retriever_msmarco = pt.BatchRetrieve(index_msmarco, wmodel="BM25")
retriever_antique = pt.BatchRetrieve(index_antique, wmodel="BM25")

C:\Users\Scott\AppData\Local\Temp\ipykernel_22644\3444506707.py:10: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  retriever_msmarco = pt.BatchRetrieve(index_msmarco, wmodel="BM25")
C:\Users\Scott\AppData\Local\Temp\ipykernel_22644\3444506707.py:11: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  retriever_antique = pt.BatchRetrieve(index_antique, wmodel="BM25")


In [25]:
# Convert qrels to DataFrames
qrels_msmarco = pd.DataFrame(msmarco.qrels_iter())
qrels_antique = pd.DataFrame(antique.qrels_iter())

# Convert queries to DataFrames
queries_msmarco = pd.DataFrame(msmarco.queries_iter())
queries_antique = pd.DataFrame(antique.queries_iter())

# Rename columns for PyTerrier compatibility
qrels_msmarco.rename(columns={"query_id": "qid", "doc_id": "docno", "relevance": "label"}, inplace=True)
qrels_antique.rename(columns={"query_id": "qid", "doc_id": "docno", "relevance": "label"}, inplace=True)

queries_msmarco.rename(columns={"query_id": "qid", "text": "query"}, inplace=True)
queries_antique.rename(columns={"query_id": "qid", "text": "query"}, inplace=True)

[INFO] Please confirm you agree to the MSMARCO data usage agreement found at <http://www.msmarco.org/dataset.aspx>
[INFO] If you have a local copy of https://msmarco.z22.web.core.windows.net/msmarcoranking/qrels.train.tsv, you can symlink it here to avoid downloading it again: C:\Users\Scott\.ir_datasets\downloads\733fb9fe12d93e497f7289409316eccf
[INFO] [starting] https://msmarco.z22.web.core.windows.net/msmarcoranking/qrels.train.tsv
[INFO] [finished] https://msmarco.z22.web.core.windows.net/msmarcoranking/qrels.train.tsv: [00:02] [10.6MB] [4.76MB/s]
[INFO] Please confirm you agree to the authors' data usage agreement found at <https://ciir.cs.umass.edu/downloads/Antique/readme.txt>
[INFO] [starting] https://ciir.cs.umass.edu/downloads/Antique/antique-test.qrel
[INFO] [finished] https://ciir.cs.umass.edu/downloads/Antique/antique-test.qrel: [00:00] [150kB] [475kB/s]
[INFO] If you have a local copy of https://msmarco.z22.web.core.windows.net/msmarcoranking/queries.tar.gz, you can symli

In [39]:
def clean_query(query):
    query = query.replace("\n", " ").replace("\r", " ")
    query = query.encode("ascii", "ignore").decode()
    query = re.sub(r"[\"`]", "", query)
    query = re.sub(r"\s+", " ", query).strip()
    query = re.sub(r"[^\w\s]", "", query)
    return query

queries_msmarco["query"] = queries_msmarco["query"].apply(clean_query)
queries_antique["query"] = queries_antique["query"].apply(clean_query)

In [40]:
queries_msmarco

,qid,query
0,121352,define extreme
1,634306,what does chattel mean on credit history
2,920825,what was the great leap forward brainly
3,510633,tattoo fixers how much does it cost
4,737889,what is decentralization process
...,...,...
808726,633855,what does canada post regulations mean
808727,1059728,wholesale lularoe price
808728,210839,how can i watch the day after
808729,908165,what to use instead of pgp in windows


In [41]:
qrels_msmarco

,qid,docno,label,iteration
0,1185869,0,1,0
1,1185868,16,1,0
2,597651,49,1,0
3,403613,60,1,0
4,1183785,389,1,0
...,...,...,...,...
532756,19285,8841362,1,0
532757,558837,4989159,1,0
532758,559149,8841547,1,0
532759,706678,8841643,1,0


In [42]:
bm25_marco = pt.terrier.Retriever(index_msmarco, wmodel="BM25")
bm25_antique = pt.terrier.Retriever(index_antique, wmodel="BM25")

In [ ]:
pt.Experiment(
    [bm25_marco],
    queries_msmarco,
    qrels_msmarco,
    eval_metrics=[RR @ 10, nDCG @ 20, MAP],
)

In [ ]:
pt.Experiment(
    [bm25_antique],
    queries_antique,
    qrels_antique,
    eval_metrics=[RR @ 10, nDCG @ 20, MAP],
)